In [1]:
import pandas as pd
import json
import re
from pprint import pprint
from itertools import chain
from collections import defaultdict
import googlemaps

In [2]:
class VenueMatcher:
    
    """
    all possibly useful Ticketek venue information is contained in two tables which we join together
    """
    TKT_VENUES = pd.read_csv('data/sample_venue_dim.csv.gz', 
                             encoding='latin-1', 
                             error_bad_lines=False, 
                             sep='\t')[['pk_venue_dim', 'venue_name', 'venue_desc', 'venue_region_desc']] \
    .join(pd.read_csv('data/sample_VenuesPowerWebAddresses.csv.gz', 
                             sep='\t', 
                             encoding='latin-1')[['venue_name', 'vcName', 'paAddressLine1', 'paAddressLine2','vcRegionName']] \
                             .set_index('venue_name'), on='venue_name', how='left').fillna('')
    
    # whenever there's no way to tell which state a venue may be in, we will go for a more popular state
    PREFERRED_STATES = 'nsw vic qld wa act sa tas nt'.split()
    
    STATES = {'nsw': 'new south wales', 
              'act': 'australian capital territory', 
              'vic': 'victoria',
              'tas': 'tasmania',
              'wa': 'western australia',
              'nt': 'northern teritory',
              'sa': 'south australia',
              'qld': 'queensland'}
    
    # now another dictionary, full to abbreviated
    STATES_ = {v: k for k, v in STATES.items()}
    
    SUBURBS = json.load(open('data/aus_suburbs_auspost_APR2017.json'))
    
    # words to catch useless venues
    BAD_WORDS = set("""games ticketek voucher circus winery cruise cirque bus events  
                        buses cruises coach reserve vineyard office memberships festival interchange""".split())
        
    
    gmaps = googlemaps.Client(**json.load(open('credentials/google.json')))
    
    def __init__(self, read_local=True):
        
        self.tkt_venues = [] if not read_local else json.load(open('data/tkt_venues.json'))
        
        print(f'working with {len(self.tkt_venues)} ticketek venues')
    
    def filter_tables(self):
        
        # ignore venues that have anything but letters in thir CODES (these are typically like ABC or DFE)
        VenueMatcher.TKT_VENUES = VenueMatcher.TKT_VENUES[VenueMatcher.TKT_VENUES['venue_name'].str.isalpha()]
        # ignore venues that have inappropriate words in their descriptions    
        VenueMatcher.TKT_VENUES = VenueMatcher.TKT_VENUES[~VenueMatcher.TKT_VENUES['venue_desc'] \
                                                          .apply(lambda _: len(VenueMatcher.BAD_WORDS & set(_.lower().split())) > 0)]
        
        print(f'filtered venues: {len(VenueMatcher.TKT_VENUES)} rows, {len(set(VenueMatcher.TKT_VENUES.pk_venue_dim))} unique keys')
        
        return self
    
    def _find_state(self, st):
        """
        find state names in string st; returns a SET of identified names
        """
        
        states_found = set()
        
        st_norm = self._normalize(st)
        
        for s in (set(VenueMatcher.STATES) | set(VenueMatcher.STATES_)):
            try:
                states_found.add(re.search(r'\b' + s + r'\b', st_norm).group(0))
            except:
                continue
                
        if states_found: # note that these may be either the full or abbreviated state names
            # return full state names to avoid rare ambiguities like WA (Australia) and WA (the US)
            return {s if s not in VenueMatcher.STATES_ else VenueMatcher.STATES_[s] for s in states_found}
        
        return states_found
    
    def _find_suburb(self, st):
        """
        find suburb names in string st; returns a set of tuples (suburb, state)
        """
        st_norm = self._normalize(st)
        
        suburbs_found = set()
        
        words_ = st_norm.split()
        
        for i, w in enumerate(words_):
            
            # take first letter of the word
            l1_ = w[0]
            
            # if any suburb names start from this letter..
            if l1_ in VenueMatcher.SUBURBS:
            
                for r in VenueMatcher.SUBURBS[l1_]:
                    
                    try:
                        suburbs_found.add((re.search(r'\b' + r['name'] + r'\b', ' '.join(words_[i:])).group(0), r['state']))
                    except:
                        continue
                        
        return suburbs_found 
    
    def find_venue_state(self):
        
        """
        look at the available Tiketek venue deacription fields and try to figure out what state the venue may
        be in; if this isn't clear, collect candidate states
        """
       
        for i, row in enumerate(VenueMatcher.TKT_VENUES.iterrows(),1):
            
            if i%100 == 0:
                print(f'processing row {i}...')
                
            this_venue = defaultdict()
        
            this_venue['name'] = self._normalize(row[1]['venue_desc'])
            this_venue['code'] = [row[1]['venue_name'].lower()]
            
            # search for state according to priority until found in one of the columns,
            # then stop
            
            for c in ['venue_desc', 'vcRegionName','venue_region_desc']:
                
                # note: set below may be empty if no states found
                candidate_states = self._find_state(self._normalize(row[1][c]))
                
                if len(candidate_states) == 1:
                    # a single candidate state
                    this_venue['state'] = candidate_states.pop()
                    break
                else: 
                    # many or no candidate states; need to find suburb 
                    for c in ['venue_desc', 'venue_region_desc']:
                        
                        # note that sub_state may be an empty set
                        suburb_state_tuples = self._find_suburb(self._normalize(row[1][c]))
                        
                        # suppose a single suburb found
                        if len(suburb_state_tuples) == 1:
                            
                            if len(candidate_states) > 0:
                                #  enough if its state is among candidate states
                                if list(suburb_state_tuples)[0][1] in candidate_states:
                                    this_venue['state'] = list(suburb_state_tuples)[0][1]
                            else:
                                # if no candidate states
                                this_venue['state'] = list(suburb_state_tuples)[0][1]
                                
                            break
                        
                        # what if more than one suburb found?
                        elif len(suburb_state_tuples) > 1:
                            
                            # suppose no candidate states
                            if not candidate_states:
                                
                                # if different suburbs in THE SAME state
                                _ = {s[1] for s in suburb_state_tuples}
                                
                                if len(_) == 1:
                                    this_venue['state'] = _.pop()
                                    break
                                    
                                else:
                                    # return the longest (in terms of the number of words in suburb name) tuple (first found)
                                    longest_sub = max(suburb_state_tuples, key=lambda x: len(x[1].split()))
                                    # only if the suburb name has AT LEAST TWO words
                                    if len(longest_sub[0].split()) > 1:
                                        this_venue['state'] = longest_sub[1]
                                    else:
                                        # simply add a list of candidate states
                                        this_venue['state_'] = list(_)
                                    break
                            else:
                                # if we have multiple candidate states AND multiple suburbs
                                for ss in suburb_state_tuples:
                                    # pick the first suburb that has its state among state candidates
                                    if ss[1] in candidate_states:
                                        this_venue['state'] = ss[1]
                                        break
                                        
            self.tkt_venues.append(this_venue)
        
        return self
    
    def merge_codes(self, on='name'):
        """
        merge Ticketek venues with multiple codes
        """
        
        venues_ = []
        # venue names already processed
        nms = set()
        
        for v in self.tkt_venues:
            
            if v[on] not in nms:
                venues_.append(v)
                nms.add(v[on])
            else:
                # this name is already available, must be under another code
                for v_ in venues_:
                    if v_[on] == v[on]:
                        v_['code'].extend(v['code'])
                        v_['code'] = list(set(v_['code']))
                        
        self.tkt_venues = venues_
            
        return self
    
    def _normalize(self, st):
        """
        normalize a string st
        """
        st = st.lower()
        # replace separators with white spaces
        st = re.sub(r'[-/_.]', ' ', st)
        # keep only letters, numbers and white spaces
        st = ''.join([l for l in st if str(l).isalnum() or str(l).isspace()])
        st = re.sub(r'\s{2,}', ' ', st)
        
        return st
    
    def _get_fields(self, res):
        """
        extract fields from a search response
        """
        up = {'place_id': res.get('place_id', None),
                                  'address': res.get('formatted_address', None),
                                  'venue_type': res.get('types', None),
                                  'coordinates': res['geometry']['location']}
        return up
        
    def get_place_id(self, local_file='data/tkt_venues.json'):
        
        """
        ask google maps to find places by name; the key here is to hopefully
        grab a place id
        """
        
        print('retrieving place ids...')
        
        if local_file:
            
            self.tkt_venues = json.load(open(local_file))
            print(f'collected {len(self.tkt_venues)} venues from the locally saved file {local_file}')
            print(f'{sum(["place_id" in v for v in self.tkt_venues])} of these already have place_ids')

        for i, v in enumerate(self.tkt_venues,1):
            
            # we want to query Google Maps for the venues that don't have a place_id yet
            
            if 'place_id' not in v:
                
                print(v['name'])
                      
                if 'state' in v:
                
                    # so we have a specific state..
                    try:
                        qr_ = self.gmaps.geocode(' '.join([v['name'], VenueMatcher.STATES[v['state']], 'australia']))
                    except:
                        print(f'no response, probably exceeded quota')
                        json.dump(self.tkt_venues, open('data/tkt_venues.json','w'))
                        break
                
                    if qr_:

                        v.update(self._get_fields(qr_[0]))
            
                else:
                
                    # problem with the state, need to consider multiple candidates
                
                    for possible_state in v['state_']:
                        
                        try:
                            qr_ = self.gmaps.geocode(' '.join([v['name'], VenueMatcher.STATES[possible_state], 'australia']))
                        except:
                            print(f'no response, probably EXCEEDED GOOGLE API QUOTA?')
                            json.dump(vm.tkt_venues, open('data/tkt_venues.json','w'))
                            break
                    
                        if qr_:
                            
                            q_top_result = None
                            
                            for r in qr_:
                                if {'political','bus_station','colloquial_area','neighborhood','transit_station'} & set(r.get('types',[])):
                                    continue
                                else:
                                    q_top_result = r
                                 
                            if q_top_result:
                                
                                for address_component in q_top_result['address_components']:
                                    # if the state we search for is in 
                                    # the result components, we say it's a suitable result
                                
                                    if address_component['short_name'].strip().lower() == possible_state:
                                        
                                        v.update(self._get_fields(q_top_result))
                                        
                                        break
        
        json.dump(self.tkt_venues, open('data/tkt_venues.json','w'))
        
        return self
    
    def get_place_details(self, local_file='data/tkt_venues.json'):
        
        """
        ask google maps for place details using a place id; 
        """
        
        print('retirieving place details...')
        
        if local_file:
            
            self.tkt_venues = json.load(open(local_file))
            print(f'collected {len(self.tkt_venues)} venues from the locally saved file {local_file}')
            print(f'{sum(["name_googlemaps" in v for v in self.tkt_venues])} of these have googlemaps name')
        
        for i, v in enumerate(self.tkt_venues, 1):
            
            if i%100 == 0:
                print(f'venue {i}: {v["name"].upper()}...')
            
            if ('place_id' in v) and ('name_googlemaps' not in v):     
                
                try:
                    place_details = self.gmaps.place(v['place_id'])['result']
                except:
                    print(f'can\'t get any place details for place_id {v["name"]}. EXCEEDED QUOTA?')
                    json.dump(self.tkt_venues, open('data/tkt_venues.json','w'))
                    return self                
                      
                try:
                    v.update({'name_googlemaps': place_details['name'].lower()})
                except:
                    print(f'no googlemap name found!')

                try:
                      v.update({'opening_hours': [d.lower() for d in place_details['opening_hours']['weekday_text']]})
                except:
                      print(f'no opening_hours found!')

                try:     
                     v.update({'rating': float(place_details['rating'])})
                except:
                     print(f'no rating found!')

                try:
                    v.update({'url_googlemaps': place_details['url']})
                except:
                    print(f'no url found!')

                try:
                    v.update({'website': place_details['website']})
                except:
                     print(f'no website found!') 
        
        
        json.dump(self.tkt_venues, open('data/tkt_venues.json','w'))
        
        return self
            
    
if __name__ == '__main__':
    
    vm = VenueMatcher()
#     .get_place_id()\
#     .get_place_details()

working with 2886 ticketek venues


In [ ]:
bads  = ['training and testing venue', 'also shed dance party events', 'no longer in use', 'brand new venue fpa',
        'geo fence event alerts 1', 'ace motorcycle tours', 'hoot jazz festival shuttle bus', 'hopman cup memberships wa',
        'eden park venue', 'geo fence event alerts 3', 'manly boat kayak hire', 'grease the mega musical national tour vic',
        'my kitchen rules rms 2012', 'balloon aloft canberra', 'hunter valley boutique wine tours lifestyle', 'museum theatrette']

In [3]:
j = 0
for v in vm.tkt_venues:
    if 'place_id' not in v:
        j += 1
        print(v)
        
print(j)

{'name': 'hunter valley boutique wine tours lifestyle', 'code': ['hbw'], 'state_': ['vic', 'sa'], 'state': 'nsw'}
{'name': 'balloon aloft canberra', 'code': ['blf'], 'state': 'act'}
{'name': 'stocklands stadium sunshine coast', 'code': ['sks'], 'state_': ['vic', 'nsw']}
{'name': 'out spirit kokoda trail', 'code': ['osk'], 'state': 'qld'}
{'name': 'ace motorcycle tours', 'code': ['amt'], 'state': 'sa'}
{'name': 'secret venue adelaide', 'code': ['sva'], 'state': 'sa'}
{'name': 'chateau elan hunter valley', 'code': ['chv'], 'state_': ['vic', 'sa']}
7


In [4]:
j = 0
for v in vm.tkt_venues:
    if 'name_googlemaps' not in v:
        j += 1
        print(v)
        
print(j)

{'name': 'hunter valley boutique wine tours lifestyle', 'code': ['hbw'], 'state_': ['vic', 'sa'], 'state': 'nsw'}
{'name': 'balloon aloft canberra', 'code': ['blf'], 'state': 'act'}
{'name': 'stocklands stadium sunshine coast', 'code': ['sks'], 'state_': ['vic', 'nsw']}
{'name': 'out spirit kokoda trail', 'code': ['osk'], 'state': 'qld'}
{'name': 'ace motorcycle tours', 'code': ['amt'], 'state': 'sa'}
{'name': 'secret venue adelaide', 'code': ['sva'], 'state': 'sa'}
{'name': 'chateau elan hunter valley', 'code': ['chv'], 'state_': ['vic', 'sa']}
7


In [ ]:
['brandon park wollongong']

In [9]:
pprint(vm.gmaps.geocode('stocklands stadium sunshine coast' + ' ' + 'queensland' )[0])

{'address_components': [{'long_name': '31',
                         'short_name': '31',
                         'types': ['street_number']},
                        {'long_name': 'Sportsmans Parade',
                         'short_name': 'Sportsmans Parade',
                         'types': ['route']},
                        {'long_name': 'Bokarina',
                         'short_name': 'Bokarina',
                         'types': ['locality', 'political']},
                        {'long_name': 'Sunshine Coast Regional',
                         'short_name': 'Sunshine Coast',
                         'types': ['administrative_area_level_2', 'political']},
                        {'long_name': 'Queensland',
                         'short_name': 'QLD',
                         'types': ['administrative_area_level_1', 'political']},
                        {'long_name': 'Australia',
                         'short_name': 'AU',
                         'types': ['country', 'poli

In [15]:
t2 = []
neg_names = set()

for v in vm.tkt_venues:
    if {'political','bus_station','colloquial_area','neighborhood','transit_station'} & set(v.get("venue_type",[])):
        t2.append(v)
        neg_names.add(v['name'])
print(len(t2))

152


In [19]:
from collections import Counter
Counter([w for n in neg_names for w in n.split()]).most_common()

[('the', 10),
 ('park', 7),
 ('hall', 7),
 ('tour', 7),
 ('festival', 6),
 ('town', 5),
 ('city', 5),
 ('events', 5),
 ('brisbane', 5),
 ('centre', 5),
 ('wa', 4),
 ('vic', 4),
 ('port', 4),
 ('event', 4),
 ('in', 4),
 ('garden', 4),
 ('transport', 4),
 ('interchange', 4),
 ('grease', 4),
 ('national', 4),
 ('tours', 4),
 ('qld', 3),
 ('venue', 3),
 ('sydney', 3),
 ('tent', 3),
 ('melbourne', 3),
 ('victoria', 3),
 ('harbour', 3),
 ('of', 3),
 ('sports', 3),
 ('geo', 3),
 ('fence', 3),
 ('alerts', 3),
 ('newcastle', 3),
 ('beach', 3),
 ('oval', 3),
 ('cup', 3),
 ('perth', 3),
 ('nsw', 3),
 ('borden', 2),
 ('auditorium', 2),
 ('mt', 2),
 ('way', 2),
 ('top', 2),
 ('station', 2),
 ('pier', 2),
 ('quay', 2),
 ('bathurst', 2),
 ('theatre', 2),
 ('live', 2),
 ('studios', 2),
 ('rally', 2),
 ('canberra', 2),
 ('supercars', 2),
 ('samsung', 2),
 ('mobile', 2),
 ('north', 2),
 ('australia', 2),
 ('3', 2),
 ('adelaide', 2),
 ('homebush', 2),
 ('bay', 2),
 ('famous', 2),
 ('spiegeltent', 2),
 ('

In [20]:
json.dump(t2, open('venues3.json','w'))

In [21]:
poli

[{'name': 'academy cinema city adelaide',
  'code': ['adc'],
  'state': 'sa',
  'place_id': 'ChIJP7Mmxcc1t2oRQMaOYlQ2AwQ',
  'address': 'Adelaide SA, Australia',
  'venue_type': ['colloquial_area', 'locality', 'political'],
  'coordinates': {'lat': -34.9284989, 'lng': 138.6007456},
  'name_googlemaps': 'adelaide',
  'url_googlemaps': 'https://maps.google.com/?q=Adelaide+SA,+Australia&ftid=0x6ab735c7c526b33f:0x4033654628ec640'},
 {'name': 'the atrium fed square',
  'code': ['atr'],
  'state': 'vic',
  'place_id': 'ChIJT8T86bZC1moR8zjqinHIOOw',
  'address': 'Swanston St, Melbourne VIC 3000, Australia',
  'venue_type': ['art_gallery',
   'establishment',
   'neighborhood',
   'point_of_interest',
   'political'],
  'coordinates': {'lat': -37.8172022, 'lng': 144.969498},
  'name_googlemaps': 'the atrium, federation square',
  'opening_hours': ['monday: open 24 hours',
   'tuesday: open 24 hours',
   'wednesday: open 24 hours',
   'thursday: open 24 hours',
   'friday: open 24 hours',
   's